In [27]:
# Number of oil fields, refineries, and receivers
oilfields = 2  # LA, SD
refineries = 2 # Dallas, Houston
receivers = 2  # New York, Chicago
numedges = oilfields * refineries + refineries * receivers + receivers # Total number of arcs

#Define the incidence matrix A for our network
A = [1  1  1  0  0  0  0  0  0  0
     0  0  0  1  1  1  0  0  0  0
    -1  0  0 -1  0  0  1  1  0  0
     0 -1  0  0 -1  0  0  0  1  1
     0  0  0  0  0  0 -1  0 -1  0
     0  0  0  0  0  0  0 -1  0 -1
     0  0 -1  0  0 -1  0  0  0  0]

#Supply (first 2) and refineries + demand (last 5 entries)
b = [400000, 600000, 0, 0, -300000, -400000, -300000]

# for 100000 barrels each
costs = [1250 10000 0 900 1320 0 470 530 450 550];



In [31]:
using JuMP, HiGHS

oil = Model(HiGHS.Optimizer)

@variable(oil, x[1:numedges] >= 0)
@constraint(oil, supplyanddemand, A*x .== b)

@objective(oil, Min, sum(costs[i]*x[i]/100000 for i in 1:numedges)) # divided by 100,000 as that was the cost ratio

print(oil)

Min 0.0125 x[1] + 0.1 x[2] + 0.009 x[4] + 0.0132 x[5] + 0.0047 x[7] + 0.0053 x[8] + 0.0045 x[9] + 0.0055 x[10]
Subject to
 supplyanddemand : x[1] + x[2] + x[3] = 400000
 supplyanddemand : x[4] + x[5] + x[6] = 600000
 supplyanddemand : -x[1] - x[4] + x[7] + x[8] = 0
 supplyanddemand : -x[2] - x[5] + x[9] + x[10] = 0
 supplyanddemand : -x[7] - x[9] = -300000
 supplyanddemand : -x[8] - x[10] = -400000
 supplyanddemand : -x[3] - x[6] = -300000
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[5] ≥ 0
 x[6] ≥ 0
 x[7] ≥ 0
 x[8] ≥ 0
 x[9] ≥ 0
 x[10] ≥ 0


In [32]:
optimize!(oil)
@show objective_value(oil)
@show value.(x)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e-03, 1e-01]
  Bound  [0e+00, 0e+00]
  RHS    [3e+05, 6e+05]
Presolving model
4 rows, 7 cols, 14 nonzeros  0s
3 rows, 7 cols, 10 nonzeros  0s
Presolve : Reductions: rows 3(-4); columns 7(-3); elements 10(-10)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.5300000000e+03 Pr: 2(700000) 0s
          3     1.0180000000e+04 Pr: 0(0) 0s
          3     1.0180000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  1.0180000000e+04
HiGHS run time      :          0.00
objective_value(oil) = 10180.0
value.(x) = [100000.0, 0.0, 300000.0, 600000.0, 0.0, 0.0, 300000.0, 400000.0, -0.0, 0.0]


10-element Vector{Float64}:
 100000.0
      0.0
 300000.0
 600000.0
      0.0
      0.0
 300000.0
 400000.0
     -0.0
      0.0